In [5]:
# @title
# -------------------------------
# Probabilities obtained from Bayesian Network
# -------------------------------
P_flood=0.55
P_infra=0.8
# -------------------------------
# Flood Preparedness - initial and goal states
# -------------------------------
init_state = {
    "action": ("start"),
    "effects": {
        ("forecast_heavy_rain", lambda P_flood: P_flood >= 0.5),
        ("flood_not_occurred", lambda P_flood: P_flood < 0.5),
        ("flood_risk_high", lambda P_flood: P_flood >= 0.5),
        ("roads_open", lambda P_flood: P_flood >= 0.5),
        ("bridges_weak", lambda P_infra: P_infra >= 0.5),
        ("boats_available", True),
        ("rescue_team_ready", True),
        ("medical_supplies_available", True),
        ("dry_areas_available", True),
        ("technicians_available", True),
        ("communication_main_up", True),
        ("backup_comm_available", True),
        ("food_supplies_stocked", True),
        ("community_alerted", False),
        ("evacuation_started", False),
        ("medical_kits_stocked", False),
        ("rescue_team_predeployed", False),
        ("routes_shifted", False)
    }
}
goal_state1 = {
    "action": ("rescue_team_work_done"),
    "effects": {
        ("people_safe", True),
        ("rescue_team_predeployed", True),
        ("rapid_response_prepared", True)
    }
}
goal_state2 = {
    "action": ("communication_team_work_done"),
    "effects": {
        ("communication_backup_operational", True),
        ("routes_shifted", True)
    }
}
goal_state3 = {
    "action": ("medical_and_supplies_work_done"),
    "effects": {
        ("medical_kits_stocked", True),
        ("supplies_ready", True)
    }
}
goal_states = {
    "goal_state1": goal_state1,
    "goal_state2": goal_state2,
    "goal_state3": goal_state3
}
# -------------------------------
# Flood Preparedness - Actions (Operators)
# -------------------------------
monitor_forecast = {
    "action": ("monitor_forecast"),
    "preconditions": {
        ("forecast_heavy_rain", True)
    },
    "effects": {
        ("flood_risk_high", True),
        ("forecast_confirmed", True)
    }
}
predeploy_rescue_team = {
    "action": ("predeploy_rescue_team"),
    "preconditions": {
        ("rescue_team_ready", True),
        ("boats_available", True),
        ("forecast_confirmed", True)
    },
    "effects": {
        ("rescue_team_predeployed", True),
        ("rapid_response_prepared", True)
    }
}
stock_medical_kits_in_advance = {
    "action": ("stock_medical_kits_in_advance"),
    "preconditions": {
        ("medical_supplies_available", True),
        ("technicians_available", True)
    },
    "effects": {
        ("medical_kits_stocked", True)
    }
}
shift_route_due_to_bridge_risk = {
    "action": ("shift_route_due_to_bridge_risk"),
    "preconditions": {
        ("bridges_weak", True),
        ("roads_open", True)
    },
    "effects": {
        ("routes_shifted", True),
        ("bridge_area_closed", True)
    }
}
evacuate_before_flood = {
    "action": ("evacuate_before_flood"),
    "preconditions": {
        ("forecast_confirmed", True),
        ("community_alerted", True),
        ("dry_areas_available", True)
    },
    "effects": {
        ("evacuation_started", True),
        ("evacuation_completed", True),
        ("people_safe", True),
        ("dry_areas_available", False)##for precendence link
    }
}
alert_community = {
    "action": ("alert_community"),
    "preconditions": {
        ("forecast_confirmed", True),
        ("community_alerted", False)
    },
    "effects": {
        ("community_alerted", True)
    }
}
establish_backup_communication = {
    "action": ("establish_backup_communication"),
    "preconditions": {
        ("backup_comm_available", True),
        ("technicians_available", True)
    },
    "effects": {
        ("communication_backup_operational", True),
        ("technicians_available", False)#for precedence link

    }
}
stockpile_food_and_medicine = {
    "action": ("stockpile_food_and_medicine"),
    "preconditions": {
        ("food_supplies_stocked", True),
        ("dry_areas_available", True)
    },
    "effects": {
        ("supplies_ready", True)
    }
}
prepare_evacuation_centers = {
    "action": ("prepare_evacuation_centers"),
    "preconditions": {
        ("dry_areas_available", True)
    },
    "effects": {
        ("evacuation_centers_ready", True)
    }
}
# -------------------------------
# Group all actions for easy access
# -------------------------------
actions = {
    "monitor_forecast": monitor_forecast,
    "predeploy_rescue_team": predeploy_rescue_team,
    "stock_medical_kits_in_advance": stock_medical_kits_in_advance,
    "shift_route_due_to_bridge_risk": shift_route_due_to_bridge_risk,
    "evacuate_before_flood": evacuate_before_flood,
    "alert_community": alert_community,
    "establish_backup_communication": establish_backup_communication,
    "stockpile_food_and_medicine": stockpile_food_and_medicine,
    "prepare_evacuation_centers": prepare_evacuation_centers,
    "start":init_state,
    "rescue_team_work_done":goal_state1,
    "communication_team_work_done":goal_state2,
    "medical_and_supplies_work_done":goal_state3
}

# -------------------------------
# causal links and precedence links
# -------------------------------
causal_links = [
    { "from_action": init_state,
      "fulfil_precondi": ("forecast_heavy_rain", True),
      "to_action": monitor_forecast
    },

    { "from_action": monitor_forecast,
      "fulfil_precondi": ("forecast_confirmed", True),
      "to_action": predeploy_rescue_team
    },

    { "from_action": init_state,
      #"fulfil_precondi": {("rescue_team_ready", True),("boats_available", True)}
      "fulfil_precondi": ("rescue_team_ready", True),
      "to_action": predeploy_rescue_team
    },

    { "from_action": init_state,
      "fulfil_precondi": ("boats_available", True),
      "to_action": predeploy_rescue_team
    },

    { "from_action": monitor_forecast,
      "fulfil_precondi": ("forecast_confirmed", True),
      "to_action": alert_community
    },

    { "from_action": init_state,
      "fulfil_precondi": ("technicians_available", True),
      "to_action": stock_medical_kits_in_advance
    },

    { "from_action": init_state,
      "fulfil_precondi": ("medical_supplies_available", True),
      "to_action": stock_medical_kits_in_advance
    },

    { "from_action": init_state,
      "fulfil_precondi": ("bridges_weak", True),
      "to_action": shift_route_due_to_bridge_risk
    },

    { "from_action": init_state,
      "fulfil_precondi": ("roads_open", True),
      "to_action": shift_route_due_to_bridge_risk
    },

    { "from_action": init_state,
      "fulfil_precondi": ("backup_comm_available", True),
      "to_action": establish_backup_communication
    },

    { "from_action": init_state,
      "fulfil_precondi": ("food_supplies_stocked", True),
      "to_action": stockpile_food_and_medicine
    },

    { "from_action": init_state,
      "fulfil_precondi": ("dry_areas_available", True),
      "to_action": prepare_evacuation_centers
    },

    { "from_action": alert_community,
      "fulfil_precondi": ("community_alerted", True),
      "to_action": evacuate_before_flood
    },

    #{ "from_action": prepare_evacuation_centers,##false link
     # "fulfil_precondi": ("dry_areas_available", True),
      #"to_action": evacuate_before_flood
    #},

    { "from_action": predeploy_rescue_team,
      "fulfil_precondi": ("rescue_team_predeployed", True),
      "to_action": goal_state1
    },

    { "from_action": establish_backup_communication,
      "fulfil_precondi": ("communication_backup_operational", True),
      "to_action": goal_state2
    },

    { "from_action": shift_route_due_to_bridge_risk,
      "fulfil_precondi": ("routes_shifted", True),
      "to_action": goal_state2
    },

    { "from_action": stock_medical_kits_in_advance,
      "fulfil_precondi": ("medical_kits_stocked", True),
      "to_action": goal_state3
    },

    { "from_action": stockpile_food_and_medicine,
      "fulfil_precondi": ("supplies_ready", True),
      "to_action": goal_state3
    },

    { "from_action": init_state,
     "fulfil_precondi": ("community_alerted", False),
     "to_action": alert_community
    },

    { "from_action": init_state,
     "fulfil_precondi": ("dry_areas_available", True),
     "to_action": evacuate_before_flood
    },

    { "from_action": init_state,
     "fulfil_precondi": ("dry_areas_available", True),
     "to_action": stockpile_food_and_medicine
    },

    { "from_action": evacuate_before_flood,
     "fulfil_precondi": ("people_safe", True),
     "to_action": goal_state1
    },

    { "from_action": monitor_forecast,
      "fulfil_precondi": ("forecast_confirmed", True),
      "to_action": evacuate_before_flood },

    { "from_action": init_state,
      "fulfil_precondi": ("technicians_available", True),
      "to_action": establish_backup_communication },

    { "from_action": predeploy_rescue_team,
      "fulfil_precondi": ("rapid_response_prepared", True),
      "to_action": goal_state1 }

]
precedence_links = [
    { "from_action": stock_medical_kits_in_advance,
      "contradict_precondi": ("technicians_available", True),
      "to_action": establish_backup_communication
    },
    { "from_action": prepare_evacuation_centers,
      "fulfil_precondi": ("dry_area_available", True),
      "to_action": evacuate_before_flood
    },
   #rightnow there is no such links but i feed two of the links can be added
]
# -------------------------------
# printing causal links and precedence links
# -------------------------------
print(causal_links)
print("\n")
print(precedence_links)



[{'from_action': {'action': 'start', 'effects': {('rescue_team_predeployed', False), ('forecast_heavy_rain', <function <lambda> at 0x7f0b53bea020>), ('backup_comm_available', True), ('food_supplies_stocked', True), ('technicians_available', True), ('rescue_team_ready', True), ('community_alerted', False), ('medical_supplies_available', True), ('medical_kits_stocked', False), ('evacuation_started', False), ('routes_shifted', False), ('flood_not_occurred', <function <lambda> at 0x7f0b53bea0c0>), ('bridges_weak', <function <lambda> at 0x7f0b53bea200>), ('communication_main_up', True), ('boats_available', True), ('dry_areas_available', True), ('flood_risk_high', <function <lambda> at 0x7f0b53bea160>), ('roads_open', <function <lambda> at 0x7f0b53be9f80>)}}, 'fulfil_precondi': ('forecast_heavy_rain', True), 'to_action': {'action': 'monitor_forecast', 'preconditions': {('forecast_heavy_rain', True)}, 'effects': {('forecast_confirmed', True), ('flood_risk_high', True)}}}, {'from_action': {'ac

In [6]:
# @title
# Improved Graphviz diagram: clusters by team (inferred from goals), LR layout, splines, saved png+pdf
import graphviz
from pathlib import Path

def visualize_pop_graphviz_better(actions, causal_links, precedence_links,
                                  filename_base="flood_POP_graph_better",
                                  prog="dot"):
    """
    Produces a cleaner Graphviz diagram:
      - LR (left-to-right) layout
      - clusters for Rescue/Communication/Medical (inferred from goal_states)
      - solid thick causal edges, dashed thin precedence edges
      - saves PNG and PDF: <filename_base>.png/.pdf
    prog: 'dot' (hierarchical), 'neato' (spring), 'sfdp' (large graphs) etc.
    """
    # infer small team mapping from goal_states: actions appearing in goal chains
    team_map = {}  # node_name -> team label
    # Rescue = goal_state1, Communication = goal_state2, Medical = goal_state3 (fallback)
    rescue_nodes = set()
    comm_nodes = set()
    med_nodes = set()
    # mark goal nodes (to cluster producers of goal facts)
    for gname, g in goal_states.items():
        if gname.lower().startswith("goal_state1") or "rescue" in gname.lower():
            # gather actions that appear in causal links to this goal
            for link in causal_links:
                if link["to_action"] is goal_state1:
                    # provider -> goal (we will mark the provider)
                    pass
    # heuristic: cluster nodes by whether their action name contains rescue/comm/med keywords
    for name in actions.keys():
        lname = name.lower()
        if any(k in lname for k in ("rescue", "evacuate", "deploy")):
            team_map[name] = "rescue"
        elif any(k in lname for k in ("comm", "communication", "route", "shift")):
            team_map[name] = "comm"
        elif any(k in lname for k in ("med", "stock", "supply", "medical")):
            team_map[name] = "medical"
        else:
            team_map[name] = "other"

    dot = graphviz.Digraph(comment="Flood POP (improved)", engine=prog)
    dot.attr(splines="true")
    dot.attr(overlap="false")
    dot.attr(rankdir="LR")
    dot.attr(fontname="Helvetica")

    # cluster colors
    cluster_colors = {"rescue": "#D6EAF8", "comm": "#D4EFDF", "medical": "#FDEBD0", "other": "#F5EEF8"}

    # build clusters
    clusters = {}
    for node, team in team_map.items():
        clusters.setdefault(team, []).append(node)

    for team, nodes in clusters.items():
        with dot.subgraph(name="cluster_" + team) as c:
            c.attr(style="filled", color=cluster_colors.get(team, "#FFFFFF"), label=team.upper())
            c.attr(fontname="Helvetica", fontsize="10")
            for n in nodes:
                c.node(n, label=n, shape="box", style="filled", fillcolor="white")

    # add edges: causal (solid bold) first
    for link in causal_links:
        try:
            from_n = [k for k, v in actions.items() if v is link["from_action"]][0]
            to_n = [k for k, v in actions.items() if v is link["to_action"]][0]
        except IndexError:
            continue
        lbl = str(link.get("fulfil_precondi", ""))
        dot.edge(from_n, to_n, label=lbl, color="black", penwidth="2.5", fontname="Helvetica", fontsize="9")

    # precedence (dashed gray)
    for link in precedence_links:
        try:
            from_n = [k for k, v in actions.items() if v is link["from_action"]][0]
            to_n = [k for k, v in actions.items() if v is link["to_action"]][0]
        except IndexError:
            continue
        lbl = str(link.get("conflict_fact", ""))
        dot.edge(from_n, to_n, label=lbl, color="gray", style="dashed", penwidth="1.5", fontname="Helvetica", fontsize="8")

    # write files
    png_path = f"{filename_base}.png"
    pdf_path = f"{filename_base}.pdf"
    dot.render(filename_base, format="png", cleanup=True)  # creates png
    dot.render(filename_base, format="pdf", cleanup=True)  # creates pdf
    print(f"Saved: {png_path} and {pdf_path}")

# Call it:
visualize_pop_graphviz_better(actions, causal_links, precedence_links)


Saved: flood_POP_graph_better.png and flood_POP_graph_better.pdf


In [7]:
from collections import defaultdict

# ===== 1. Define actions and ordering constraints (from your POP plan) =====

actions = [
    "start",
    "shift_route_due_to_bridge_risk",
    "monitor_forecast",
    "stock_medical_kits_in_advance",
    "stockpile_food_and_medicine",
    "alert_community",
    "predeploy_rescue_team",
    "establish_backup_communication",
    "medical_and_supplies_work_done",
    "evacuate_before_flood",
    "communication_team_work_done",
    "rescue_team_work_done",
]

# (u, v) means: u must come BEFORE v
edges = [
    ("start", "stock_medical_kits_in_advance"),
    ("predeploy_rescue_team", "rescue_team_work_done"),
    ("monitor_forecast", "predeploy_rescue_team"),
    ("start", "predeploy_rescue_team"),
    ("establish_backup_communication", "communication_team_work_done"),
    ("start", "shift_route_due_to_bridge_risk"),
    ("evacuate_before_flood", "rescue_team_work_done"),
    ("alert_community", "evacuate_before_flood"),
    ("shift_route_due_to_bridge_risk", "communication_team_work_done"),
    ("monitor_forecast", "evacuate_before_flood"),
    ("start", "establish_backup_communication"),
    ("start", "evacuate_before_flood"),
    ("start", "medical_and_supplies_work_done"),
    ("start", "rescue_team_work_done"),
    ("stockpile_food_and_medicine", "medical_and_supplies_work_done"),
    ("stock_medical_kits_in_advance", "medical_and_supplies_work_done"),
    ("stockpile_food_and_medicine", "evacuate_before_flood"),
    ("start", "stockpile_food_and_medicine"),
    ("stock_medical_kits_in_advance", "establish_backup_communication"),
    ("start", "monitor_forecast"),
    ("monitor_forecast", "alert_community"),
    ("start", "communication_team_work_done"),
    ("start", "alert_community"),
]

# ===== 2. Function to generate ALL valid linearizations (topological sorts) =====

def all_topological_sorts(actions, edges, limit=None):
    """
    actions: iterable of action names
    edges: iterable of (u, v) meaning u must come before v
    limit: optional int -> stop after this many linearizations (None = no limit)

    Returns: list of linearizations (each is a list of actions)
    """
    actions = list(actions)

    # Build adjacency list and indegree map
    adj = defaultdict(list)
    indeg = {a: 0 for a in actions}
    for u, v in edges:
        adj[u].append(v)
        indeg[v] += 1

    all_orders = []
    n = len(actions)

    def backtrack(order, indeg_local):
        # If we have a full ordering, record it
        if len(order) == n:
            all_orders.append(order.copy())
            if limit is not None and len(all_orders) >= limit:
                return True  # signal to stop early
            return False

        # Choose any action with indegree 0 that is not yet in the order
        for a in actions:
            if a not in order and indeg_local[a] == 0:
                order.append(a)
                changed = []
                for v in adj[a]:
                    indeg_local[v] -= 1
                    changed.append(v)

                should_stop = backtrack(order, indeg_local)

                # Undo changes
                for v in changed:
                    indeg_local[v] += 1
                order.pop()

                if should_stop:
                    return True

        return False

    backtrack([], indeg.copy())
    return all_orders

# ===== 3. Run and inspect results =====

# WARNING: for this plan, there are 43881 linearizations.
# If you don't want to keep them all, use `limit=` in the function call above.

orders = all_topological_sorts(actions, edges)  # or all_topological_sorts(actions, edges, limit=10)

print("Number of valid linearizations:", len(orders))

# Print the first few linearizations nicely
max_to_show = 5
for i, order in enumerate(orders[:max_to_show], start=1):
    print(f"\nLinearization {i}:")
    for step, act in enumerate(order, start=1):
        print(f"  {step}. {act}")


Number of valid linearizations: 43881

Linearization 1:
  1. start
  2. shift_route_due_to_bridge_risk
  3. monitor_forecast
  4. stock_medical_kits_in_advance
  5. stockpile_food_and_medicine
  6. alert_community
  7. predeploy_rescue_team
  8. establish_backup_communication
  9. medical_and_supplies_work_done
  10. evacuate_before_flood
  11. communication_team_work_done
  12. rescue_team_work_done

Linearization 2:
  1. start
  2. shift_route_due_to_bridge_risk
  3. monitor_forecast
  4. stock_medical_kits_in_advance
  5. stockpile_food_and_medicine
  6. alert_community
  7. predeploy_rescue_team
  8. establish_backup_communication
  9. medical_and_supplies_work_done
  10. evacuate_before_flood
  11. rescue_team_work_done
  12. communication_team_work_done

Linearization 3:
  1. start
  2. shift_route_due_to_bridge_risk
  3. monitor_forecast
  4. stock_medical_kits_in_advance
  5. stockpile_food_and_medicine
  6. alert_community
  7. predeploy_rescue_team
  8. establish_backup_commu